# SQLite DB: Storing and Retrieving Vectors

In [1]:
# !pip install sqlite3

import sqlite3
import numpy as np

In [2]:
# create a connection to the SQLite DB
conn = sqlite3.connect("vector-db.db")
# Create a cursor object to execute SQL Commands
cursor = conn.cursor()

In [34]:
# Create a table for vector data
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS vectors (
    id INTEGER PRIMARY KEY,
    vector BLOB NOT NULL
)
"""
)

In [35]:
# generate some sample vectors
vect1 = np.array([21.2, 3.4, 2.1, 0.8])
vect2 = np.array([2.7, 1.5, 3.9, 2.3])
vect3 = np.array([21.7, 1.5, 32.9, 2.3])
vect4 = np.array([43.7, 1.5, 3.9, 2.3])
vect5 = np.array([12.7, 1.5, 4.9, 2.3])
vect6 = np.array([12.7, 6.5, 3.9, 2.3])

In [36]:
vect1.tobytes()  # numpy array to bytestream

b'3333335@333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?'

In [37]:
# Insert vector data into table
cursor.execute(
    "INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect1.tobytes()),)
)

OperationalError: database is locked

In [8]:
cursor.execute(
    "INSERT INTO vectors (vector) VALUES (?)", (sqlite3.Binary(vect2.tobytes()),)
)

In [9]:
# Retreive data

cursor.execute("SELECT vector FROM vectors")

In [10]:
rows = cursor.fetchall()

In [11]:
rows

[(b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?',),
 (b'333333\xf3?333333\x0b@\xcd\xcc\xcc\xcc\xcc\xcc\x00@\x9a\x99\x99\x99\x99\x99\xe9?',),
 (b'\x9a\x99\x99\x99\x99\x99\x05@\x00\x00\x00\x00\x00\x00\xf8?333333\x0f@ffffff\x02@',)]

In [12]:
vector = np.frombuffer(rows[0][0], dtype=np.float64)

In [13]:
vector

array([1.2, 3.4, 2.1, 0.8])

In [14]:
vectors = []
for row in rows:
    vector = np.frombuffer(row[0], dtype=np.float64)
    vectors.append(vector)

In [15]:
vectors

[array([1.2, 3.4, 2.1, 0.8]),
 array([1.2, 3.4, 2.1, 0.8]),
 array([2.7, 1.5, 3.9, 2.3])]

# Vector Similarity Search (VSS)

In [20]:
query_vect = np.array([14.0, 3.2, 2.0, 0.5])

In [21]:
cursor.execute(
    """

SELECT vector FROM vectors ORDER BY abs(vector - ?) ASC

""",
    (sqlite3.Binary(query_vect.tobytes()),),
)

In [22]:
res = cursor.fetchone()  # finding the top one

In [23]:
np.frombuffer(res[0], dtype=np.float64)  # most similar vector

array([2.7, 1.5, 3.9, 2.3])

In [29]:
conn.commit()

In [30]:
conn.close()